In [1]:
# Reference: https://www.kaggle.com/dansbecker/transfer-learning
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib
import os
import cv2
import random
import keras

from glob import glob
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# loading the directories 
training_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Train'
test_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Test'
folders=os.listdir("C:/Users/avadh/Desktop/CMPE 295A/Dataset/Temp/Semi_Train")
print(folders)
#test_dir = '../input/fruits/fruits-360_dataset/fruits-360/test-multiple_fruits/'

['Chiffon', 'Jeans', 'Lace_Dress', "Men's_Shirts", 'Skirt']


In [3]:
size=64,64

In [4]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

In [5]:
image_train=[]
image_labels=[]
image_names=[]

for folder in folders:
    for each in os.listdir(os.path.join(training_dir,folder)):
        if each.endswith('jpg'):
            image_names.append(os.path.join(training_dir,folder,each))
            image_labels.append(folder)
            img=cv2.imread(os.path.join(training_dir,folder,each))
            img_in=cv2.resize(img,size)
            image_train.append(img_in)
        else:
            continue


In [6]:
union_list=list(zip(image_train,image_labels))
random.shuffle(union_list)
train,labels=zip(*union_list)
X=np.array(train)
Y=np.array(labels)

In [7]:
X=X/255.0


In [8]:
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,num_classes=len(folders))

In [9]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.1)


In [10]:
vgg16_model=keras.applications.vgg16.VGG16(include_top=False,input_shape=(64,64,3))

In [11]:
model=Sequential()

for layers in vgg16_model.layers:
    model.add(layers)
    
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))

for layer in vgg16_model.layers:
    layer.trainable=False

for layers in vgg16_model.layers[-2:]:
    layers.trainable=True

In [12]:
model.add(Dense(len(folders),activation='softmax'))

In [13]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 256)       295168    
__________

In [14]:
optimizer=RMSprop(lr=0.0001)   #,rho=0.9,epsilon=1e-08,decay=0.0)
model.compile(optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False) 

datagen.fit(X_train)

In [16]:
history=model.fit_generator(datagen.flow(X_train,Y_train,batch_size=10),epochs=5,validation_data=(X_val,Y_val),
                          verbose=1,steps_per_epoch=X_train.shape[0]/10)


Epoch 1/5
536/535 [==============================] - 495s 924ms/step - loss: 1.1237 - acc: 0.5610 - val_loss: 1.1743 - val_acc: 0.6158
Epoch 2/5
536/535 [==============================] - 504s 941ms/step - loss: 0.9231 - acc: 0.6541 - val_loss: 1.0185 - val_acc: 0.5973
Epoch 3/5
536/535 [==============================] - 488s 910ms/step - loss: 0.8456 - acc: 0.6875 - val_loss: 0.9194 - val_acc: 0.7047
Epoch 4/5
536/535 [==============================] - 547s 1s/step - loss: 0.7950 - acc: 0.7033 - val_loss: 0.8779 - val_acc: 0.6762
Epoch 5/5
536/535 [==============================] - 737s 1s/step - loss: 0.7573 - acc: 0.7267 - val_loss: 0.9322 - val_acc: 0.7064


In [17]:
#Reference: https://machinelearningmastery.com/save-load-keras-deep-learning-models/

from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("Dataset/MyVersion/VGG16.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
